In [1]:
from os import walk
import re
import sys
import os
import numpy as np
import operator
import math

In [2]:
unique_words = set()
train_data = {'0': {},'1':{} ,'2':{} ,'3':{}}
exp_per_class = {'0': 0,'1': 0 ,'2':0 ,'3':0}

In [3]:
stop_words = set(open("stopwords.txt","r").read().replace('\n', ' ').split(' ')) 

In [4]:
input_path = 'op_spam_training_data'

In [5]:
def text_cleaning(text):
    cleaned_txt = re.sub('[^a-z\s]+', ' ', text, flags=re.IGNORECASE)
    return re.sub('(\s+)', ' ', cleaned_txt).lower().replace("'", "")

In [6]:
def category(file_name):
    text = open(file_name, 'r').read()
    
    #Cleaning and splitting the data
    cleaned_text = text_cleaning(text)
    word_arr = cleaned_text.split(' ')
    
    #Removing the stop words
    word_arr = [each for each in word_arr if each not in stop_words]     
    if 'negative_polarity' in file_name:
        dict_index = "0"
    elif 'negative_polarity/truthful' in file_name:
        dict_index = "1"
    elif 'positive_polarity/truthful' in file_name:
        dict_index = "2"
    elif 'positive_polarity/deceptive' in file_name:
        dict_index = "3"
    else:
        print("Error, no dict_index")
        
    exp_per_class[dict_index] += 1

    for each in word_arr:
        unique_words.add(each)
        if each in train_data[dict_index]:
            train_data[dict_index][each] += 1
        else:
            train_data[dict_index][each] = 1

In [7]:
for (dirpath, dirnames, filenames) in walk(next(os.walk(input_path))[0]):
    for file in filenames:
        file_name = os.path.join(dirpath, file)
        search_str ='fold2|fold3|fold4'
        if bool(re.search(search_str, file_name)):
            category(file_name)

In [8]:
total_reviews = sum(exp_per_class.values())
prob_class = {}
for each in exp_per_class.keys():
    prob_class[each] = math.log(exp_per_class[each] / total_reviews)

In [9]:
mod_v = len(unique_words)

train_keys = {}      
for key in train_data.keys():
    train_keys[key] = sum(train_data[key].values())

In [11]:
prob_score = {'0': {},'1':{} ,'2':{} ,'3':{}}
for word in unique_words:
    for key in train_data:
        count_word = (train_data[key][word] + 1) if word in train_data[key] else 1
        prob_word = count_word / (train_keys[key] + mod_v + 1)
        prob_score[key][word] = math.log(prob_word)

In [12]:
result  = {"pro_each_class" :prob_class, 'probability_score' : prob_score}
with open('nbmodel.txt', 'w') as file:
    file.write(str(result).replace("'", "\""))
    file.close()